In [19]:
import os
import argparse
from PIL import Image
import torch
torch.set_printoptions(profile="default")
from torch import nn, optim
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter
from torch.autograd import Variable

parser = argparse.ArgumentParser('Caltech101_AlexNet')
parser.add_argument('--ObjectCategories_dir', type=str, default='101_ObjectCategories')
parser.add_argument('--seed', type=int, default=20) # 数据集划分随机种子
parser.add_argument('--batch_size', type=int, default=10)
parser.add_argument('--img_size', type=int, default=224) # 通过序号选择用哪一个多层感知，用来对比效果
parser.add_argument('--nn', type=str, default='0') # 通过序号选择用哪一个神经网络，用来对比效果
parser.add_argument('--gpu', type=int, default=0) # -1时为CPU
parser.add_argument('--niters', type=int, default=50) # 训练的epoch数
parser.add_argument('--model_dir', type=str, default='./model/AlexNet.pth') # 最佳模型存储路径
parser.add_argument('--tensorboard_dir', type=str, default='')
args = parser.parse_args(args=['--batch_size', '256', '--niters', '50', '--nn', '2', '--tensorboard_dir', ''])

device = torch.device('cpu' if args.gpu == -1 else 'cuda:' + str(args.gpu))


class Caltech101Dataset(Dataset):
    def __init__(self, args, split='train'):
        data_dir = args.ObjectCategories_dir
        categories = os.listdir(data_dir)
        categories.remove('BACKGROUND_Google')
        
        self.transforms = transforms.Compose([
            transforms.Resize((args.img_size, args.img_size)),
            transforms.ToTensor(),])
        
        X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []
        
        for i, cat in enumerate(categories):
            cat_dir = os.path.join(data_dir, cat)
            X = []
            y = []
            for file in os.listdir(cat_dir):
                img_path = os.path.join(cat_dir, file)
                img = self.transforms(Image.open(img_path).convert('RGB'))
                X.append(img)
                y.append(i)
            
            # stratify=y 表示在分割数据时要保持标签 y 的比例，这意味着训练集和测试集中每个类别的样本比例与原始数据集中相同。
            X_train_val, X_test_cat, y_train_val, y_test_cat = train_test_split(X, y, test_size=0.1, 
                                                                                stratify=y, random_state=args.seed)
            X_train_cat, X_val_cat, y_train_cat, y_val_cat = train_test_split(X_train_val, y_train_val, 
                                                                              test_size=0.1111, stratify=y_train_val, 
                                                                              random_state=args.seed)
            
            X_train.extend(X_train_cat) # X_train_cat 中的所有元素都将被添加到 X_train 列表的末尾
            y_train.extend(y_train_cat)
            X_val.extend(X_val_cat)
            y_val.extend(y_val_cat)
            X_test.extend(X_test_cat)
            y_test.extend(y_test_cat)

        if split == 'train':
            self.X = X_train
            self.y = y_train
        elif split == 'val':
            self.X = X_val
            self.y = y_val
        elif split == 'test':
            self.X = X_test
            self.y = y_test

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
  

class AlexNet_0(nn.Module):
    def __init__(self, num_classes=101, init_weights=True):   
        super(AlexNet_0, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[96, 55, 55] 
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[96, 27, 27] 
            nn.Conv2d(96, 256, kernel_size=5, padding=2),           # output[256, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[256, 13, 13]
            nn.Conv2d(256, 384, kernel_size=3, padding=1),          # output[256, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),          # output[384, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),          # output[256, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            #全连接
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') #何教授方法
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  #正态分布赋值
                nn.init.constant_(m.bias, 0)

class AlexNet_1(nn.Module):
    def __init__(self, num_classes=101, init_weights=True):   
        super(AlexNet_1, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27] kernel_num为原论文一半
            nn.Conv2d(48, 128, kernel_size=5, padding=2),           # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 13, 13]
            nn.Conv2d(128, 128, kernel_size=3, padding=1),          # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            #全链接
            nn.Linear(128 * 6 * 6, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') #何教授方法
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  #正态分布赋值
                nn.init.constant_(m.bias, 0)

class AlexNet_2(nn.Module):
    def __init__(self, num_classes=101, init_weights=True):   
        super(AlexNet_2, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27] kernel_num为原论文一半
            nn.Conv2d(48, 128, kernel_size=5, padding=2),           # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 13, 13]
            nn.Conv2d(128, 128, kernel_size=3, padding=1),          # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            #全链接
            nn.Linear(128 * 6 * 6, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') #何教授方法
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  #正态分布赋值
                nn.init.constant_(m.bias, 0)

class AlexNet_3(nn.Module):
    def __init__(self, num_classes=101, init_weights=True):   
        super(AlexNet_3, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27] kernel_num为原论文一半
            nn.Conv2d(48, 128, kernel_size=5, padding=2),           # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 13, 13]
            nn.Conv2d(128, 64, kernel_size=3, padding=1),          # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            #全链接
            nn.Linear(64 * 6 * 6, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') #何教授方法
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  #正态分布赋值
                nn.init.constant_(m.bias, 0)
                

class AlexNet_4(nn.Module):
    def __init__(self, num_classes=101, init_weights=True):   
        super(AlexNet_4, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27] kernel_num为原论文一半
            nn.Dropout(p=0.5),
            nn.Conv2d(48, 64, kernel_size=5, padding=2),           # output[64, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),    
            nn.Conv2d(64, 64, kernel_size=3, padding=1),          # output[64, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[64, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            #全链接
            nn.Linear(64 * 6 * 6, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') #何教授方法
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  #正态分布赋值
                nn.init.constant_(m.bias, 0)

                           

if __name__ == '__main__':
    # 加载数据，分割成训练、验证、测试集（保持类别比例）
    train_data = Caltech101Dataset(args, split='train')
    torch.manual_seed(args.seed)
    train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True) # 打乱训练集训练，防过拟合
    val_data = Caltech101Dataset(args, split='val')
    val_loader = DataLoader(val_data, batch_size=args.batch_size, shuffle=False) 
    test_data = Caltech101Dataset(args, split='test')
    test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=False) 
    print ('Data loaded.')

    # 选择模型
    model_name = 'AlexNet_' + args.nn
    model_name = globals()[model_name]
    model = model_name().to(device)
    
    writer = SummaryWriter(args.tensorboard_dir)
    fake_img = torch.randn(20, 3, 224, 224).to(device)
    writer.add_graph(model, (Variable(fake_img),), True)
        
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    scheduler = ReduceLROnPlateau(optimizer, 'min')

    # 训练
    best_accur = 0
    for epoch in range(args.niters):
        train_loss = 0.0
        for image, true_y in train_loader: 
            # print (image.size()) 
            optimizer.zero_grad() 
            true_y = true_y.to(device)
            pred_y = model(image.to(device)).to(device)    # 得到预测值
            # print ('true_y', true_y, 'pred_y:',pred_y)
            # exit()
            loss = criterion(pred_y, true_y)    # 计算两者的平均损失
            # 反向传播和优化
            loss.backward()
            optimizer.step() 
            train_loss += loss.item() * image.size(0) # image.size(0) = batch_size, 将每batch的平均损失变成累计损失
        train_loss = train_loss / len(train_loader.dataset)     
        scheduler.step(loss)
      
        # 验证
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            correct_top5 = 0
            total = 0
            for image, true_y in val_loader: 
                pred_y = model(image.to(device)).to(device)
                true_y = true_y.to(device)
                loss = criterion(pred_y, true_y).to(device) 
                val_loss += loss.item() * image.size(0) 
                total += image.size(0)
                # print ('\n\n', pred_y, torch.argmax(pred_y, dim=1))
                correct += (true_y == torch.argmax(pred_y, dim=1)).sum().item()
                top5_pred_y = torch.topk(pred_y, k=5, dim=1).indices
                correct_top5 += (top5_pred_y == true_y.unsqueeze(dim=1)).any(dim=1).sum().item()
            val_loss = val_loss / len(val_loader.dataset)
            val_accur = correct / total
            val_accur_top5 = correct_top5 / total
            
            if val_accur > best_accur:
                best_accur = val_accur
                torch.save(model, args.model_dir)

        print('Epoch:  {} \tTrain Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Top1 Accuracy: {:.2%} \nValidation Top5 Accuracy: {:.2%}'
            .format(epoch + 1, train_loss, val_loss, val_accur, val_accur_top5))
            
        # 训练过程参数存进tensorboard
        writer.add_scalar('train_loss', train_loss, epoch)
        writer.add_scalar('val_accuracy', val_accur, epoch)
        writer.add_scalar('val_top5_accuracy', val_accur_top5, epoch)
        
    writer.close()
    print ('Save tensornoard.')       
    
    
    # 测试
    with torch.no_grad():
        test_loss = 0.0
        correct = 0
        correct_top5 = 0
        total = 0
        for image, true_y in test_loader: 
            pred_y = model(image.to(device)).to(device)
            true_y = true_y.to(device)
            loss = criterion(pred_y, true_y).to(device) 
            test_loss += loss.item() * image.size(0) 
            total += image.size(0)
            # print ('\n\n', pred_y, torch.argmax(pred_y, dim=1))
            correct += (true_y == torch.argmax(pred_y, dim=1)).sum().item()
            top5_pred_y = torch.topk(pred_y, k=5, dim=1).indices
            correct_top5 += (top5_pred_y == true_y.unsqueeze(dim=1)).any(dim=1).sum().item()
        test_loss = test_loss / len(test_loader.dataset)
        test_accur = correct / total
        test_accur_top5 = correct_top5 / total
        
    print ('Test Loss: {:.6f} \tTesting Accuracy: {:.2%} \tTesting Top5 Accuracy: {:.2%}'
           .format(test_loss, test_accur, test_accur_top5))

Data loaded.
graph(%self.1 : __torch__.torch.nn.modules.module.___torch_mangle_359.Module,
      %input.1 : Float(20, 3, 224, 224)):
  %196 : __torch__.torch.nn.modules.module.___torch_mangle_358.Module = prim::GetAttr[name="classifier"](%self.1)
  %187 : __torch__.torch.nn.modules.module.___torch_mangle_353.Module = prim::GetAttr[name="features"](%self.1)
  %212 : Tensor = prim::CallMethod[name="forward"](%187, %input.1)
  %142 : int = prim::Constant[value=1]() # /tmp/ipykernel_883/148130232.py:195:0
  %143 : int = prim::Constant[value=-1]() # /tmp/ipykernel_883/148130232.py:195:0
  %input.10 : Float(20, 4608) = aten::flatten(%212, %142, %143) # /tmp/ipykernel_883/148130232.py:195:0
  %213 : Tensor = prim::CallMethod[name="forward"](%196, %input.10)
  return (%213)



/home/ma-user/anaconda3/envs/PyTorch-1.4/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type AlexNet_2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch:  1 	Train Loss: 4.296320 	Validation Loss: 4.214108 	Validation Top1 Accuracy: 14.05% 
Validation Top5 Accuracy: 30.09%
Epoch:  2 	Train Loss: 3.843490 	Validation Loss: 3.549787 	Validation Top1 Accuracy: 25.11% 
Validation Top5 Accuracy: 37.28%
Epoch:  3 	Train Loss: 3.103064 	Validation Loss: 2.954982 	Validation Top1 Accuracy: 35.84% 
Validation Top5 Accuracy: 52.77%
Epoch:  4 	Train Loss: 2.542129 	Validation Loss: 2.488135 	Validation Top1 Accuracy: 43.69% 
Validation Top5 Accuracy: 62.94%
Epoch:  5 	Train Loss: 2.062861 	Validation Loss: 2.207846 	Validation Top1 Accuracy: 49.23% 
Validation Top5 Accuracy: 67.48%
Epoch:  6 	Train Loss: 1.712056 	Validation Loss: 2.222185 	Validation Top1 Accuracy: 51.22% 
Validation Top5 Accuracy: 69.25%
Epoch:  7 	Train Loss: 1.486735 	Validation Loss: 2.046207 	Validation Top1 Accuracy: 53.54% 
Validation Top5 Accuracy: 72.46%
Epoch:  8 	Train Loss: 1.232423 	Validation Loss: 2.003794 	Validation Top1 Accuracy: 57.30% 
Validation Top5 A

In [18]:
%reload_ext ma_tensorboard
%ma_tensorboard  --logdir './runs' --port 8000

Starting TensorBoard on port 8000 (pid 16216, Use "!kill 16216" to kill it.)